In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import os
import sys  
sys.path.insert(0, r'C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data')

In [62]:
import warnings
from Signals_Processing import *
from Process import *
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import ensemble, feature_selection
import pickle

In [63]:
os.chdir('C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\')
os.getcwd()

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system'

In [64]:
"""Load Data from Folder"""

folder = os.getcwd() + "\\Training Models\\"
participant = "P3_testing"
path = folder + participant
path

'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P3_testing'

In [65]:
EEG_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P3\EEGbaseline.csv', sep=";", index_col=0)

In [66]:
df_baseline = pd.read_csv(r'C:\Users\Rodrigo\Desktop\PhD\loop-system\Training Models\P3\baseline.csv', sep=";", index_col=0)

In [68]:
imp = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P3\\imp.pkl", "rb"))
scaler = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P3\\scaler.pkl", "rb"))
rfe = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P3\\rfe.pkl", "rb"))
model = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P3\\model.pkl", "rb"))

In [69]:
EEG_baseline

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,EEG_30_gamma,EEG_30_theta,EEG_31_alpha,EEG_31_betha,EEG_31_gamma,EEG_31_theta,EEG_32_alpha,EEG_32_betha,EEG_32_gamma,EEG_32_theta
0,2.526250e-09,-1.070726e-07,-2.052582e-07,-2.400296e-08,6.667488e-10,4.978328e-09,1.019009e-08,-6.498092e-09,4.108139e-08,3.119779e-08,...,-0.000013,-0.000008,5.076837e-08,2.863135e-07,7.426426e-07,-7.383299e-08,4.340428e-08,1.248841e-07,5.381564e-07,9.078149e-07


In [70]:
df_baseline

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1
0,78.563429,68.961686,92.302564,4.258938,0.765965,0.650036,0.870048,41.6439,24.6541,14,...,0.267207,391.0,53.3736,0.136505,29.0,16.0,39.02439,70.731707,0.11283,79.822911


In [71]:
os.chdir(path)
(
    users,
    EEG_epochs,
    EEG_filtered,
    Signals_epochs,
    Signals,
    EEG_dict,
    features_signals,
    features_EEG,
    features_epochs,
    features_epochs_EEG,
) = ({}, {}, {}, {}, {}, {}, {}, {}, {}, {})

for root, dirs, files in os.walk(path):
    for fname in files:
        if fname.endswith(".xdf"):
            users[fname] = Run_files(fname)

In [72]:
users

{'P3_testing.xdf': [{'info': defaultdict(list,
               {'name': ['PsychoPy Ratings'],
                'type': ['Ratings'],
                'channel_count': ['2'],
                'channel_format': ['string'],
                'source_id': ['Example'],
                'nominal_srate': ['0.000000000000000'],
                'version': ['1.100000000000000'],
                'created_at': ['56626.16392360000'],
                'uid': ['c6c93779-8913-4bc9-a6fe-25e44ed183c8'],
                'session_id': ['default'],
                'hostname': ['DESKTOP-8HVM7BE'],
                'v4address': [None],
                'v4data_port': ['16574'],
                'v4service_port': ['16574'],
                'v6address': [None],
                'v6data_port': ['16574'],
                'v6service_port': ['16574'],
                'desc': [None],
                'stream_id': 5,
                'effective_srate': 0}),
   'footer': {'info': defaultdict(list,
                {'first_timestamp'

In [73]:
Opensignals_fs = 100
EEG_fs = 250
resolution = 16

In [74]:
sensors = ["ECG", "EDA", "RESP"]
data = {}
for user in users.keys():
    data[user.split(".")[0]] = Load_Data(
        users[user],
        "OpenSignals",
        "openvibeSignal",
        "PsychoPy Markers",
        "PsychoPy Markers",
        sensors,
    )

    data[user.split(".")[0]]["Markers"].pop(0)
    data[user.split(".")[0]]["Markers"].pop(-1)
    data[user.split(".")[0]]["Markers Timestamps"].pop(0)
    data[user.split(".")[0]]["Markers Timestamps"].pop(-1)

onset, offset, videos, valence, arousal = ({}, {}, {}, {}, {})
for user in data.keys():
    valence[user] = data[user]["Valence"]
    arousal[user] = data[user]["Arousal"]
    onset[user], offset[user], videos[user] = getMarkers(
        data[user]["Markers"], data[user]["Markers Timestamps"]
    )

onset_index = {}
offset_index = {}
onset_index_EEG = {}
offset_index_EEG = {}

for user in data.keys():
    onset_index[user], offset_index[user] = getMarkersIndex(
        onset[user], offset[user], data[user]["Signals"]["Time"]
    )
    onset_index_EEG[user], offset_index_EEG[user] = getMarkersIndex(
        onset[user], offset[user], np.array(data[user]["EEG"]["Time"])
    )

events_diff = {}

for keys in onset.keys():
    events_diff[keys] = CalculateEventsDiff(onset[keys], offset[keys])

In [75]:
"""EEG Processing"""

EEG_filtered = filterEEG(data, EEG_fs)
EEG_filtered = getEEGChannels(EEG_filtered)
EEG_epochs = getEpochs(EEG_filtered, onset_index_EEG, events_diff, EEG_fs)
EEG_dict = getVideosDict(EEG_epochs, videos)
features_EEG = getEEGBands(EEG_dict, EEG_fs)

In [76]:
features_EEG

{'P3_testing': {'EMDB/Horror/1002.avi': {'EEG_1': {'alpha': 6.197997534427298e-09,
    'betha': 1.0028825560786643e-07,
    'gamma': 1.6763975305660222e-07,
    'theta': 5.243820633633573e-09},
   'EEG_2': {'alpha': 1.0452530434499976e-09,
    'betha': 6.564230009101417e-09,
    'gamma': 9.711658930280678e-09,
    'theta': 2.7214742931763223e-09},
   'EEG_3': {'alpha': 1.7315846216773736e-08,
    'betha': 3.2043271077782025e-08,
    'gamma': 2.0491473581715242e-08,
    'theta': 2.9171167292270625e-08},
   'EEG_4': {'alpha': 8.425027604038853e-10,
    'betha': 4.959781222841808e-09,
    'gamma': 6.745889105367717e-09,
    'theta': 1.9926106420093662e-09},
   'EEG_5': {'alpha': 8.006578352926e-10,
    'betha': 2.144896731536019e-09,
    'gamma': 2.37094809814824e-09,
    'theta': 2.869408353639224e-09},
   'EEG_6': {'alpha': 8.051495493833125e-10,
    'betha': 6.741072881385105e-09,
    'gamma': 9.549275376642506e-09,
    'theta': 1.3948857502302807e-09},
   'EEG_7': {'alpha': 7.31854282

In [77]:
EEG_list = EEG_baseline.loc[0].values.flatten().tolist()

In [78]:
Signals_epochs = getSignalsEpochs(data, onset_index, events_diff, Opensignals_fs)
Signals = getVideosDict(Signals_epochs, videos)
features_signals = getFeatures(Signals, Opensignals_fs, resolution)

C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 128 is greater than input length  = 5, using nperseg = 5
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data\lib\sensors.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ratio = round(lf_norm / hf_norm, 2)
C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 128 is greater than input length  = 5, using nperseg = 5
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\Rodrigo\Desktop\PhD\Study1\Physiological_Data\lib\sensors.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ratio = round(lf_norm / hf_norm, 2)
C:\ProgramData\Anaconda3\envs\Loop System\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 128 is greater than input length  = 5, using nperseg = 5
 

In [79]:
columns = list()
category, video = [], []

for key in features_epochs_EEG.keys():
    user = key
    print(user)
videos = list(features_EEG[user].keys())
for epochs in features_EEG[user][videos[0]].keys():
    for bands in features_EEG[user][videos[0]][epochs].keys():
        columns.append(epochs + "_" + bands)
df_EEG = pd.DataFrame(columns=columns)

for users in features_EEG.keys():
    temp_df = pd.DataFrame(columns=columns)

    for i, videos in enumerate(features_EEG[users].keys()):
        if videos != "baseline":
            temp_list = list()
            for channel in features_EEG[users][videos].keys():
                for band in features_EEG[users][videos][channel].keys():
                    temp_list.append(
                        features_EEG[users][videos][channel][band]
                    )
            temp_df.loc[i] = list(np.array(temp_list)-np.array(EEG_list))
            category.append(videos.split("/")[1])
            video.append(videos.split("/")[2])

    df_EEG = pd.concat([df_EEG, temp_df], ignore_index=True)
df_EEG["Category"] = category
df_EEG["Video"] = video

In [80]:
df_EEG

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,EEG_31_alpha,EEG_31_betha,EEG_31_gamma,EEG_31_theta,EEG_32_alpha,EEG_32_betha,EEG_32_gamma,EEG_32_theta,Category,Video
0,3.671748e-09,2.073609e-07,3.728980e-07,2.924678e-08,3.785042e-10,1.585902e-09,-4.784354e-10,9.219566e-09,-2.376554e-08,8.454805e-10,...,1.615131e-07,1.752383e-07,-2.295276e-07,7.853717e-07,1.148056e-07,2.007747e-07,-2.102369e-07,2.405540e-07,Horror,1002.avi
1,4.148377e-09,1.983283e-07,3.448641e-07,3.036172e-08,7.056131e-10,1.109582e-09,-5.212887e-10,9.562637e-09,-3.134334e-08,-1.346406e-08,...,2.113034e-07,2.111454e-07,-1.207575e-07,7.678165e-07,1.370617e-07,2.307234e-07,-1.299175e-07,-1.763193e-07,Horror,1008.avi
2,9.768032e-10,1.548883e-07,2.700529e-07,2.928338e-08,3.518813e-10,9.677450e-10,-7.418655e-10,9.476482e-09,-3.126144e-08,-1.519193e-08,...,1.678014e-07,2.778067e-07,9.819372e-08,8.304388e-07,9.983775e-08,2.491295e-07,-1.992844e-08,-2.586840e-07,Horror,1003.avi
3,3.760229e-09,1.957659e-07,3.399159e-07,3.037691e-08,4.525113e-10,1.974038e-09,1.083180e-09,9.125998e-09,-2.619724e-08,-7.419862e-09,...,1.961051e-07,2.413563e-07,-4.897475e-08,7.359948e-07,1.598902e-07,2.446280e-07,-1.091193e-07,-1.159748e-07,Horror,1006.avi
4,8.445318e-09,1.895722e-07,3.294907e-07,3.297439e-08,6.868884e-10,2.085099e-09,9.032903e-10,9.801275e-09,-2.609982e-08,-9.926510e-09,...,1.587525e-07,2.074274e-07,-1.613982e-07,7.073617e-07,1.127535e-07,1.831640e-07,-2.350665e-07,-3.268777e-07,Horror,1005.avi
5,2.312815e-03,1.793929e-03,4.959926e-04,6.169682e-03,2.314171e-03,1.794352e-03,4.951157e-04,6.177330e-03,2.325712e-03,1.811416e-03,...,2.246418e-03,1.828521e-03,5.344108e-04,5.829173e-03,2.411228e-03,1.991518e-03,5.963389e-04,6.196302e-03,Social Negative,3005.avi
6,4.303600e-03,3.343712e-03,9.311988e-04,1.102927e-02,4.306177e-03,3.344505e-03,9.312678e-04,1.100524e-02,4.327748e-03,3.374311e-03,...,4.259692e-03,3.408402e-03,9.832916e-04,1.075009e-02,4.432315e-03,3.588347e-03,1.052979e-03,1.109264e-02,Social Negative,3002.avi
7,6.889464e-09,2.166718e-07,4.004921e-07,8.564260e-07,3.724408e-09,1.595261e-09,-3.313262e-10,9.210105e-07,-2.743299e-08,-1.025146e-08,...,1.735761e-07,9.435025e-08,-3.616878e-07,1.501895e-06,1.705693e-07,1.472196e-07,-3.004175e-07,1.014177e-06,Social Negative,3008.avi
8,4.381326e-09,2.508326e-07,4.773283e-07,3.109684e-08,4.308887e-10,2.689587e-09,3.205309e-09,9.596029e-09,-2.729938e-08,-5.771057e-09,...,1.576127e-07,1.401296e-07,-2.977039e-07,7.382714e-07,1.229756e-07,1.470189e-07,-3.309901e-07,-1.285815e-07,Social Negative,3003.avi
9,4.747368e-09,2.327464e-07,4.370190e-07,3.247607e-08,2.973067e-10,1.806238e-09,3.244945e-09,9.448384e-09,-2.937600e-08,-1.143489e-08,...,1.432715e-07,1.195673e-07,-2.569230e-07,6.177044e-07,1.120573e-07,1.381644e-07,-3.634272e-07,-5.247221e-07,Social Negative,3006.avi


In [81]:
category, videos = [], []

for key in features_signals.keys():
    user = key
video = list(features_signals[user].keys())
df = pd.DataFrame(columns=features_signals[user][video[0]].columns)
for users in features_signals.keys():
    for epoch in features_signals[users].keys():
        if epoch != "baseline":
            df = pd.concat([df, features_signals[users][epoch].sub(df_baseline)], ignore_index=True)
            category.append(epoch.split("/")[1])
            videos.append(epoch.split("/")[2])
df["Category"] = category
df["Video"] = videos

In [82]:
df

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,Category,Video
0,0.189807,2.449104,-5.367689,-1.060781,-0.002834,0.040136,-0.029839,-10.6211,-4.5992,-12,...,85.9908,0.090657,-26.0,-13.0,35.975610,4.268293,NaN,158.111254,Horror,1002.avi
1,-2.111112,-0.013367,-6.609654,0.116246,0.021431,0.050138,0.000169,3.7564,-1.1310,-11,...,124.5384,0.178383,-25.0,-13.0,20.975610,9.268293,NaN,95.825412,Horror,1008.avi
2,-2.340964,-5.147706,-1.415987,2.158431,0.026680,0.010127,0.070185,23.7606,7.4669,-6,...,145.2948,0.126284,-26.0,-13.0,35.975610,4.268293,NaN,78.705215,Horror,1003.avi
3,-0.655822,0.788340,-6.609675,-0.443138,0.006027,0.050139,-0.009834,-3.8091,-0.6158,-12,...,-38.5476,-0.093027,-24.0,-13.0,-1.524390,-8.231707,NaN,-33.967289,Horror,1006.avi
4,-1.623336,1.609214,-8.989696,-1.060074,0.015207,0.070141,-0.019840,-9.2622,-5.2586,-12,...,139.3644,0.318602,-24.0,-12.0,27.642276,12.601626,NaN,98.048953,Horror,1005.avi
5,-2.280869,2.449051,-11.241728,-2.178862,0.021170,0.090149,-0.029839,-20.1430,-5.8855,-13,...,-21.4977,-0.050004,-22.0,-14.0,-14.024390,16.768293,NaN,-36.978508,Social Negative,3005.avi
6,0.509293,2.449686,-4.088516,-0.231314,-0.005216,0.030128,-0.029846,-3.2024,-4.9888,-11,...,83.0256,0.191377,-25.0,-12.0,18.118467,-13.588850,NaN,100.038378,Social Negative,3002.avi
7,1.026222,-0.796653,-7.816326,-1.272312,-0.010972,0.060139,0.010168,-11.5411,-8.0226,-14,...,-34.8411,-0.087217,-25.0,-13.0,-1.524390,-20.731707,NaN,-49.153009,Social Negative,3008.avi
8,-0.074780,3.310131,-7.815792,-1.120671,-0.000310,0.060134,-0.039849,-11.2575,-5.1361,-13,...,38.5476,0.087693,-23.0,-10.0,46.689895,14.982578,NaN,104.264025,Social Negative,3003.avi
9,-2.628652,-8.976656,-11.241713,0.056673,0.027090,0.090149,0.130201,9.6783,-3.4356,-13,...,25.2042,0.064976,-23.0,-10.0,46.689895,14.982578,NaN,-21.136172,Social Negative,3006.avi


In [83]:
"""Concatenate Dataframes"""
columns = df.columns[: (len(df.columns) - 2)]
columns_EEG = df_EEG.columns[: (len(df_EEG.columns) - 2)]
full_dataframe = pd.concat([df_EEG[columns_EEG], df], axis=1)
full_columns = full_dataframe.columns[: (len(full_dataframe.columns)-2)]

In [84]:
full_dataframe

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,Category,Video
0,3.671748e-09,2.073609e-07,3.728980e-07,2.924678e-08,3.785042e-10,1.585902e-09,-4.784354e-10,9.219566e-09,-2.376554e-08,8.454805e-10,...,85.9908,0.090657,-26.0,-13.0,35.975610,4.268293,NaN,158.111254,Horror,1002.avi
1,4.148377e-09,1.983283e-07,3.448641e-07,3.036172e-08,7.056131e-10,1.109582e-09,-5.212887e-10,9.562637e-09,-3.134334e-08,-1.346406e-08,...,124.5384,0.178383,-25.0,-13.0,20.975610,9.268293,NaN,95.825412,Horror,1008.avi
2,9.768032e-10,1.548883e-07,2.700529e-07,2.928338e-08,3.518813e-10,9.677450e-10,-7.418655e-10,9.476482e-09,-3.126144e-08,-1.519193e-08,...,145.2948,0.126284,-26.0,-13.0,35.975610,4.268293,NaN,78.705215,Horror,1003.avi
3,3.760229e-09,1.957659e-07,3.399159e-07,3.037691e-08,4.525113e-10,1.974038e-09,1.083180e-09,9.125998e-09,-2.619724e-08,-7.419862e-09,...,-38.5476,-0.093027,-24.0,-13.0,-1.524390,-8.231707,NaN,-33.967289,Horror,1006.avi
4,8.445318e-09,1.895722e-07,3.294907e-07,3.297439e-08,6.868884e-10,2.085099e-09,9.032903e-10,9.801275e-09,-2.609982e-08,-9.926510e-09,...,139.3644,0.318602,-24.0,-12.0,27.642276,12.601626,NaN,98.048953,Horror,1005.avi
5,2.312815e-03,1.793929e-03,4.959926e-04,6.169682e-03,2.314171e-03,1.794352e-03,4.951157e-04,6.177330e-03,2.325712e-03,1.811416e-03,...,-21.4977,-0.050004,-22.0,-14.0,-14.024390,16.768293,NaN,-36.978508,Social Negative,3005.avi
6,4.303600e-03,3.343712e-03,9.311988e-04,1.102927e-02,4.306177e-03,3.344505e-03,9.312678e-04,1.100524e-02,4.327748e-03,3.374311e-03,...,83.0256,0.191377,-25.0,-12.0,18.118467,-13.588850,NaN,100.038378,Social Negative,3002.avi
7,6.889464e-09,2.166718e-07,4.004921e-07,8.564260e-07,3.724408e-09,1.595261e-09,-3.313262e-10,9.210105e-07,-2.743299e-08,-1.025146e-08,...,-34.8411,-0.087217,-25.0,-13.0,-1.524390,-20.731707,NaN,-49.153009,Social Negative,3008.avi
8,4.381326e-09,2.508326e-07,4.773283e-07,3.109684e-08,4.308887e-10,2.689587e-09,3.205309e-09,9.596029e-09,-2.729938e-08,-5.771057e-09,...,38.5476,0.087693,-23.0,-10.0,46.689895,14.982578,NaN,104.264025,Social Negative,3003.avi
9,4.747368e-09,2.327464e-07,4.370190e-07,3.247607e-08,2.973067e-10,1.806238e-09,3.244945e-09,9.448384e-09,-2.937600e-08,-1.143489e-08,...,25.2042,0.064976,-23.0,-10.0,46.689895,14.982578,NaN,-21.136172,Social Negative,3006.avi


In [85]:
X = np.array(full_dataframe[full_columns])
Y = np.array(full_dataframe[["Category"]])

In [86]:
X = imp.transform(X)
X = scaler.transform(X)
X = rfe.transform(X)

prediction = model.predict(X)
print(prediction)
print(full_dataframe[["Category"]])
from sklearn import metrics
print("Accuracy = " + str(metrics.accuracy_score(np.array(full_dataframe[["Category"]]),prediction)*100)+"%")

['Horror' 'Horror' 'Horror' 'Social Negative' 'Horror' 'Social Negative'
 'Horror' 'Social Negative' 'Horror' 'Social Negative']
          Category
0           Horror
1           Horror
2           Horror
3           Horror
4           Horror
5  Social Negative
6  Social Negative
7  Social Negative
8  Social Negative
9  Social Negative
Accuracy = 70.0%


In [87]:
output = pd.read_csv(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\P3\output.csv', sep=",", index_col=False)
# features = pd.read_csv(r'C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\P1\final_features.csv', sep=",", index_col=False)

In [88]:
output

,Variable,True Value,Prediction
0,Category,Horror,Social Negative
1,Category,Horror,Social Negative
2,Category,Horror,Social Negative
3,Category,Horror,Social Negative
4,Category,Horror,Social Negative
...,...,...,...
1046,Category,Social Negative,Social Negative
1047,Category,Social Negative,Social Negative
1048,Category,Social Negative,Social Negative
1049,Category,Social Negative,Social Negative


In [89]:
print("Real Accuracy = " + str(metrics.accuracy_score(np.array(output["True Value"]),np.array(output["Prediction"]))*100)+"%")

Real Accuracy = 49.00095147478592%


In [10]:
features

,EEG_1_alpha,EEG_1_betha,EEG_1_gamma,EEG_1_theta,EEG_2_alpha,EEG_2_betha,EEG_2_gamma,EEG_2_theta,EEG_3_alpha,EEG_3_betha,...,RRV_CVSD,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1
0,0.000002,7.930818e-07,0.000810,0.000007,0.000023,0.000004,0.019973,0.000075,3.330531e-05,1.263925e-05,...,-0.214109,-18.0,-28.9107,-0.072311,-28.0,-19.0,-45.238095,-23.015873,NaN,-64.733583
1,0.000002,7.888573e-07,0.000813,0.000007,0.000023,0.000005,0.019944,0.000085,3.340273e-05,1.267303e-05,...,-0.215185,-18.0,-34.8411,-0.088923,-29.0,-19.0,-45.238095,-34.126984,NaN,-64.993177
2,0.000002,7.871184e-07,0.000820,0.000008,0.000023,0.000004,0.019944,0.000100,3.337410e-05,1.257313e-05,...,-0.212341,-18.5,-42.2541,-0.109600,-29.0,-19.0,-45.238095,-28.571429,NaN,-63.951388
3,0.000002,7.817731e-07,0.000828,0.000007,0.000022,0.000004,0.019956,0.000074,3.297560e-05,1.231116e-05,...,-0.212341,-18.5,-42.2541,-0.109600,-29.0,-19.0,-45.238095,-28.571429,NaN,-63.951388
4,0.000002,7.893067e-07,0.000835,0.000008,0.000022,0.000004,0.019957,0.000100,3.317275e-05,1.225755e-05,...,-0.211728,-17.5,-41.5128,-0.107701,-29.0,-19.0,-45.238095,-28.571429,NaN,-63.771889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,0.000006,2.195289e-06,0.002339,0.000019,0.000018,0.000003,0.008400,0.000047,4.621875e-07,1.515247e-07,...,-0.196652,-6.0,-42.2541,-0.111713,-30.0,-17.0,-23.015873,-45.238095,NaN,-59.773022
1931,0.000006,2.209341e-06,0.002385,0.000016,0.000018,0.000003,0.008521,0.000050,4.660621e-07,1.532851e-07,...,-0.191835,-6.0,-42.2541,-0.111713,-30.0,-17.0,-23.015873,-45.238095,NaN,-58.470722
1932,0.000006,2.244610e-06,0.002437,0.000018,0.000018,0.000003,0.008647,0.000052,4.716830e-07,1.557388e-07,...,-0.206949,-5.0,-48.1845,-0.127904,-31.0,-18.0,-32.738095,-53.571429,NaN,-62.333989
1933,0.000006,2.273048e-06,0.002494,0.000018,0.000018,0.000003,0.008785,0.000045,4.789261e-07,1.577481e-07,...,-0.209765,-5.0,-48.1845,-0.127904,-31.0,-18.0,-32.738095,-53.571429,NaN,-63.112704


In [11]:
from sklearn import metrics
print("Real Accuracy = " + str(metrics.accuracy_score(np.array(output["True Value"]),np.array(output["Prediction"]))*100)+"%")
# print("Simulated Accuracy = " + str(metrics.accuracy_score(np.array(simulate["True Value"]),np.array(simulate["Prediction"]))*100)+"%")

Real Accuracy = 46.40826873385013%


In [13]:
imp = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\imp.pkl", "rb"))
scaler = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\scaler.pkl", "rb"))
rfe = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\rfe.pkl", "rb"))
model = pickle.load(open("C:\\Users\\Rodrigo\\Desktop\\PhD\\loop-system\\Training Models\\P0\\model.pkl", "rb"))

In [14]:
X = np.array(features)
X = imp.transform(X)
X = scaler.transform(X)
X = rfe.transform(X)

prediction = model.predict(X)
print("Accuracy = " + str(metrics.accuracy_score(np.array(output["True Value"]),prediction)*100)+"%")

Accuracy = 46.40826873385013%


In [ ]:
from sklearn import metrics
X = np.array(full_dataframe)
​
X = imp.transform(X)
X = scaler.transform(X)
X = rfe.transform(X)
​
prediction = (model.predict(X))
print("Accuracy = " + str(metrics.accuracy_score(categories,prediction)*100)+"%")
Accuracy = 48.57547265190577%